In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import pandas as pd
import seaborn as sns
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import neighbors, datasets
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from scipy.spatial import ConvexHull
from tqdm import tqdm
import random
plt.style.use('ggplot')
import pickle
from sklearn import tree
from sklearn.tree import export_graphviz
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
%matplotlib inline
from sklearn.impute import SimpleImputer

In [3]:
def getAuc(X,y,test_size=0.25,max_depth=None,n_estimators=100,
           minsplit=4,FPR=[],TPR=[],VERBOSE=False, USE_ONLY=None):
    '''
        get AUC given training data X, with target labels y
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    CLASSIFIERS=[DecisionTreeClassifier(max_depth=max_depth, min_samples_split=minsplit,class_weight='balanced'),
                RandomForestClassifier(n_estimators=n_estimators,
                                       max_depth=max_depth,min_samples_split=minsplit,class_weight='balanced'),
                ExtraTreesClassifier(n_estimators=n_estimators,
                                     max_depth=max_depth,min_samples_split=minsplit,class_weight='balanced'),
                AdaBoostClassifier(n_estimators=n_estimators),
                GradientBoostingClassifier(n_estimators=n_estimators,max_depth=max_depth),
                svm.SVC(kernel='rbf',gamma='scale',class_weight='balanced',probability=True)]

    if USE_ONLY is not None:
        if isinstance(USE_ONLY, (list,)):
            CLASSIFIERS=[CLASSIFIERS[i] for i in USE_ONLY]
        if isinstance(USE_ONLY, (int,)):
            CLASSIFIERS=CLASSIFIERS[USE_ONLY]

    for clf in CLASSIFIERS:
        clf.fit(X_train,y_train)
        y_pred=clf.predict_proba(X_test)
        #print(X_test,y_pred)
        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred[:,1], pos_label=1)
        auc=metrics.auc(fpr, tpr)
        if VERBOSE:
            print(auc)

        FPR=np.append(FPR,fpr)
        TPR=np.append(TPR,tpr)
    points=np.array([[a[0],a[1]] for a in zip(FPR,TPR)])
    hull = ConvexHull(points)
    x=np.argsort(points[hull.vertices,:][:,0])
    auc=metrics.auc(points[hull.vertices,:][x,0],points[hull.vertices,:][x,1])
    return auc,CLASSIFIERS


def saveFIG(filename='tmp.pdf',AXIS=False):
    '''
        save fig for publication
    '''
    import pylab as plt
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    plt.margins(0,0)
    if not AXIS:
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.savefig(filename,dpi=300, bbox_inches = 'tight',
                pad_inches = 0,transparent=False) 
    return

In [4]:
def getCoverage(model,verbose=True):
    '''
        return how many distinct items (questions)
        are used in the model set.
        This includes the set of questions being
        covered by all forms that may be 
        generated by the model set
    '''
    FS=[]
    for m in model:
        for count in range(len(m.estimators_)):
            clf=m.estimators_[count]
            fs=clf.tree_.feature[clf.tree_.feature>0]
            FS=np.array(list(set(np.append(FS,fs))))
    if verbose:
        print("Number of items used: ", FS.size)
    return FS

def getConfusion(X,y,test_size=0.25,max_depth=None,n_estimators=100,
           minsplit=4,CONFUSION={},VERBOSE=False, USE_ONLY=None,target_names = None):
    '''
        get AUC given training data X, with target labels y
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    CLASSIFIERS=[DecisionTreeClassifier(max_depth=max_depth, min_samples_split=minsplit),
                RandomForestClassifier(n_estimators=n_estimators,class_weight='balanced',
                                       max_depth=max_depth,min_samples_split=minsplit),
                ExtraTreesClassifier(n_estimators=n_estimators,class_weight='balanced',
                                     max_depth=max_depth,min_samples_split=minsplit),
                AdaBoostClassifier(n_estimators=n_estimators),
                GradientBoostingClassifier(n_estimators=n_estimators,max_depth=max_depth),
                svm.SVC(kernel='rbf',gamma='scale',class_weight='balanced',probability=True)]

    if USE_ONLY is not None:
        if isinstance(USE_ONLY, (list,)):
            CLASSIFIERS=[CLASSIFIERS[i] for i in USE_ONLY]
        if isinstance(USE_ONLY, (int,)):
            CLASSIFIERS=CLASSIFIERS[USE_ONLY]

    for clf in CLASSIFIERS:
        clf.fit(X_train,y_train)
        y_pred=clf.predict(X_test)
        print(y_test,y_pred)
        cmat=confusion_matrix(y_test, y_pred)
        acc=accuracy_score(y_test, y_pred)
        
        CONFUSION[clf]=cmat
        
        if VERBOSE:
            print(classification_report(y_test, y_pred, target_names=target_names))
            print('Confusion MAtrix:\n', cmat)
            print(' ')
            print('Accuracy:', acc)

        
    return CONFUSION,acc

In [5]:
df=pd.read_csv('bsnip.csv',index_col=0)
df.head()

,project,Biotype,panss_p1,panss_p2,panss_p3,panss_p4,panss_p5,panss_p6,panss_p7,panss_n1,...,young_9,young_10,young_11,sfs_setotal,sfs_ictotal,sfs_ipcptotal,sfs_ipcctotal,sfs_retotal,sfs_prototal,sfs_oetotal
subject_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,2,4.0,3.0,4.0,1.0,3.0,2.0,1.0,4.0,...,0.0,1.0,2.0,9.0,8.0,27.0,NaN,25.0,45.0,5.0
4,1.0,5,3.0,1.0,3.0,1.0,2.0,2.0,1.0,3.0,...,0.0,0.0,0.0,7.0,8.0,19.0,NaN,13.0,9.0,4.0
53,1.0,2,4.0,3.0,5.0,2.0,1.0,3.0,1.0,5.0,...,0.0,1.5,0.0,10.0,4.0,24.0,31.0,10.0,17.0,0.0
73,1.0,2,6.0,5.0,6.0,4.0,5.0,6.0,6.0,1.0,...,2.0,1.0,4.0,7.0,7.0,21.0,18.0,15.0,11.0,10.0
78,1.0,1,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,...,0.0,0.0,0.0,9.0,9.0,36.0,39.0,13.0,12.0,1.0


In [6]:
# 3 is HC
df.Biotype.value_counts()

1    703
5    589
2    532
3     76
Name: Biotype, dtype: int64

In [30]:
#df=df[df['Biotype']==3]
df=df.dropna()
df0=df

In [31]:
#df=df0[df0.Biotype.isin([1,5])]
df=df0
X=df.iloc[:,2:].values
y=df.Biotype.values#.astype(str)
y5=[(int(x)==5)+0 for x in y ]
y1=[(int(x)==1)+0 for x in y ]
y2=[(int(x)==2)+0 for x in y ]


In [51]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.1)
y5=[(int(x)==5)+0 for x in y_train_ ]
y1=[(int(x)==1)+0 for x in y_train_ ]
y2=[(int(x)==2)+0 for x in y_train_ ]


In [52]:
ACC=[]
CLFh={}
for run in tqdm(np.arange(500)):
    auc,CLFS=getAuc(X_train_,y5,test_size=0.2,max_depth=10,n_estimators=2,
               minsplit=2,VERBOSE=False, USE_ONLY=[2])
    ACC=np.append(ACC,auc)
    if auc > 0.75:
        CLFh[auc]=CLFS
#sns.distplot(ACC)
print(np.median(ACC))
CLFstar5=CLFh[np.array([k for k in CLFh.keys()]).max()][0]

100%|██████████| 500/500 [00:04<00:00, 124.19it/s]

0.7446740585586017


In [53]:
ACC=[]
CLFh={}
for run in tqdm(np.arange(500)):
    auc,CLFS=getAuc(X_train_,y1,test_size=0.2,max_depth=10,n_estimators=2,
               minsplit=2,VERBOSE=False, USE_ONLY=[2])
    ACC=np.append(ACC,auc)
    if auc > 0.75:
        CLFh[auc]=CLFS
#sns.distplot(ACC)
print(np.median(ACC))
CLFstar1=CLFh[np.array([k for k in CLFh.keys()]).max()][0]

100%|██████████| 500/500 [00:04<00:00, 122.65it/s]

0.6962035173471344


In [54]:
ACC=[]
CLFh={}
for run in tqdm(np.arange(500)):
    auc,CLFS=getAuc(X_train_,y2,test_size=0.2,max_depth=10,n_estimators=2,
               minsplit=2,VERBOSE=False, USE_ONLY=[2])
    ACC=np.append(ACC,auc)
    if auc > 0.75:
        CLFh[auc]=CLFS
#sns.distplot(ACC)
print(np.median(ACC))
CLFstar2=CLFh[np.array([k for k in CLFh.keys()]).max()][0]

100%|██████████| 500/500 [00:03<00:00, 125.42it/s]

0.6804109809150742


In [47]:

y_pred5=CLFstar5.predict(X_test_)
y_pred1=CLFstar1.predict(X_test_)
y_pred2=CLFstar2.predict(X_test_)

#print(y_pred5)
#print(y_pred1)
Y=[]
for (i,j,k) in zip(y_pred1,y_pred5):
    if i==0 and j==1:
        k=5
    if i==1 and j==0:
        k=1
    if i==0 and j==0:
        k=2
    if i==1 and j==1:
        k=1
    Y=np.append(Y,k)   
Y

array([5., 1., 1., 2., 1., 1., 2., 5., 1., 1., 1., 5., 2., 2., 1., 2., 1.,
       2., 1., 1., 1., 5., 1., 5., 1., 5., 1., 1., 5., 2., 2., 5., 1., 2.,
       1., 2., 2., 2., 1., 2., 5., 1., 5., 1., 1., 1., 1., 5., 1., 1., 2.,
       2., 2., 5., 2., 1., 2., 1., 5., 1., 5., 5., 2., 1., 2., 2., 2., 5.,
       2., 1., 1., 5., 5., 1., 5., 5., 1., 2., 2., 1., 2., 5., 5., 2., 5.,
       2., 2., 2., 1., 5., 5., 2., 5., 2., 5., 5., 1., 5., 1., 5., 1., 5.,
       2., 1., 5., 2., 1., 2., 5., 2., 2., 2., 5., 1., 1., 2., 1., 5., 1.,
       1., 2., 5., 1., 1., 1., 1., 2., 5., 1., 1., 1., 2., 1., 2., 2., 1.,
       1., 5., 1., 2., 2., 2., 2., 5., 5., 1., 5., 2., 2., 1., 1., 1., 5.,
       2., 5.])

In [48]:
y_test_

array([1, 1, 1, 2, 1, 1, 1, 5, 1, 1, 2, 2, 1, 1, 1, 5, 1, 5, 1, 1, 2, 5,
       1, 5, 1, 2, 2, 1, 1, 1, 5, 5, 1, 1, 5, 1, 1, 1, 1, 2, 2, 2, 5, 1,
       1, 1, 5, 5, 1, 1, 2, 5, 1, 2, 1, 1, 2, 5, 5, 2, 5, 1, 5, 1, 1, 5,
       1, 2, 5, 1, 1, 2, 1, 1, 5, 5, 5, 2, 1, 1, 5, 1, 1, 2, 5, 2, 1, 2,
       2, 5, 5, 1, 5, 2, 2, 5, 1, 2, 1, 2, 2, 5, 2, 2, 5, 2, 2, 2, 1, 2,
       2, 2, 5, 1, 1, 5, 2, 1, 1, 5, 1, 5, 1, 1, 5, 2, 2, 1, 5, 5, 5, 5,
       1, 5, 2, 5, 2, 5, 1, 1, 5, 5, 2, 5, 5, 2, 2, 2, 2, 2, 1, 1, 2, 1,
       5])

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_, Y)

0.5161290322580645

In [ ]:
from scipy import interpolate
from scipy.interpolate import interp1d
auc_=[]
ROC={}
fpr_ = np.linspace(0, 1, num=20, endpoint=True)
for run in np.arange(1000):
    clf=CLFstar
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
    y_pred=clf.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred[:,1], pos_label=1)
    f = interp1d(fpr, tpr)
    auc_=np.append(auc_,metrics.auc(fpr_, f(fpr_)))
    ROC[metrics.auc(fpr, tpr)]={'fpr':fpr_,'tpr':f(fpr_)}
sns.distplot(auc_)
auc_.mean()